In [1]:
import os
import csv
import json
import dateutil.parser

In [17]:
def get_right_db(db_list, db_name, db_search):
    # print(db_name)
    runtime_list = []
    for db in db_list:
        if db_search in db:
            with open(workdir + db_dir + "/" + db) as csv_file:
                csv_reader = list(csv.reader(csv_file, delimiter=','))[1:]
                for row in csv_reader:
                    # print("\t", row[-2], row[-1])
                    dict_0 = json.loads(row[3])
                    event_time = dict_0["Records"][0]["eventTime"]
                    dt = dateutil.parser.isoparse(event_time)
                    event_time = dt.timestamp()
                    runtime = round(float(row[-1]) - float(event_time), 3)
                    # print("\t", str(runtime))
                    runtime_list.append(runtime)
    return runtime_list

## Runtime check

In [18]:
workdir = "/home/scrapjack/PycharmProjects/OSCAR-P/runs-results/annotations-test/Database/"

database_dirs = sorted(os.listdir(workdir))
database_dirs.remove("output")

csv_header = "blur_1, blur_2, mask_1, mask_2, dummy_1, dummy_2, total"
csv_content = []

for db_dir in database_dirs:
    # print(db_dir)
    
    db_list = sorted(os.listdir(workdir + db_dir))
    # print(db_list)
    
    blurs = get_right_db(db_list, "blurry-faces", "blur")
    masks = get_right_db(db_list, "mask-detector", "mask")
    dummies = get_right_db(db_list, "dummy", "dummy")
    
    times = blurs + masks + dummies
    
    total = 0
    for t in times:
        total += t
    
    times += [round(total, 3)]
    
    csv_content.append(str(times))
    
print("Done!")

Done!


In [19]:
with open(workdir + "output/summary.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_content:
        file.write(row[1:-1] + "\n")
print("Done!")

Done!


## Constraints check

In [20]:
workdir = "/home/scrapjack/PycharmProjects/OSCAR-P/runs-results/annotations-test/Database/"

database_dirs = sorted(os.listdir(workdir))
database_dirs.remove("output")

csv_header = "blur, mask, dummy, blur+mask, mask+dummy, global"
csv_content = []

for db_dir in database_dirs:
    # print(db_dir)
    
    db_list = sorted(os.listdir(workdir + db_dir))
    # print(db_list)
    
    blurs = get_right_db(db_list, "blurry-faces", "blur")
    blur = blurs[0] + blurs[1]
    masks = get_right_db(db_list, "mask-detector", "mask")
    mask = masks[0] + masks[1]
    dummies = get_right_db(db_list, "dummy", "dummy")
    dummy = dummies[0] + dummies[1]
    
    blur_mask = blur + mask
    mask_dummy = mask + dummy
    global_time = blur + mask + dummy
    
    times = [blur, mask, dummy, blur_mask, mask_dummy, global_time]
    
    for i in range(len(times)):
        times[i] = round(times[i], 3)
    
    csv_content.append(str(times))
    
print("Done!")

Done!


In [21]:
with open(workdir + "output/constraints_check.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_content:
        file.write(row[1:-1] + "\n")
print("Done!")

Done!


## Constraints validation

In [35]:
workdir = "/home/scrapjack/PycharmProjects/OSCAR-P/runs-results/annotations-test/Database/"

database_dirs = sorted(os.listdir(workdir))
database_dirs.remove("output")

csv_header = "blur, mask, dummy, blur+mask, mask+dummy, global"
csv_content = []

for db_dir in database_dirs:
    # print(db_dir)
    
    db_list = sorted(os.listdir(workdir + db_dir))
    # print(db_list)
    
    blurs = get_right_db(db_list, "blurry-faces", "blur")
    blur = blurs[0] + blurs[1]
    masks = get_right_db(db_list, "mask-detector", "mask")
    mask = masks[0] + masks[1]
    dummies = get_right_db(db_list, "dummy", "dummy")
    dummy = dummies[0] + dummies[1]
    
    blur_mask = blur + mask
    mask_dummy = mask + dummy
    global_time = blur + mask + dummy
    
    checks = [0] * 6
    times = [blur, mask, dummy, blur_mask, mask_dummy, global_time]
    constraints = [5, 5, 20, 10, 20, 20]
    
    for i in range(6):
        if times[i] > constraints[i]:
            checks[i] = 1
    
    csv_content.append(str(checks))
    
print("Done!")

Done!


In [34]:
with open(workdir + "output/constraints_validation.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_content:
        file.write(row[1:-1] + "\n")
print("Done!")

Done!


## Merge influxdb.csv

In [14]:
def correct_row(row, db_search, i):
    dict_0 = json.loads(row[3])
    event_time = dict_0["Records"][0]["eventTime"]
    
    dt = dateutil.parser.isoparse(event_time)
    event_time = dt.timestamp()
    
    row[3] = str(event_time)
    row[0] = "video_" + str(i)
    row[2] = db_search
    
    #print(row)
    
    row_string = ','.join([elem for elem in row])
    # print(row_string)
    
    return row_string

In [3]:
def get_csv_content(db_list, db_search, i):
    lines = []
    for db in db_list:
        if db_search in db:
            with open(workdir + db_dir + "/" + db) as csv_file:
                # lines += csv_file.readlines()[1:]
                csv_rows = list(csv.reader(csv_file, delimiter=','))[1:]
                for row in csv_rows:
                    lines += [correct_row(row, db_search, i)]
    return lines

In [15]:
workdir = "/home/scrapjack/PycharmProjects/OSCAR-P/runs-results/annotations-test/Database/"

database_dirs = sorted(os.listdir(workdir))
database_dirs.remove("output")

csv_header = "uuid,resource_id,component_name,start_time_job,start_time_func,end_time_func"
csv_content = []

for i in range(len(database_dirs)):
    db_dir = database_dirs[i]
    # print(db_dir)
    
    db_list = sorted(os.listdir(workdir + db_dir))
    # print(db_list)
    
    blurs = get_csv_content(db_list, "blur", i)
    masks = get_csv_content(db_list, "mask", i)
    dummies = get_csv_content(db_list, "dummy", i)
    
    lines = blurs + masks + dummies
    # print(len(lines))
    
    csv_content += lines

# print(len(csv_content))
print("Done!")

Done!


In [16]:
with open(workdir + "output/influxdb.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_content:
        file.write(row + "\n")
print("Done!")

Done!


## Parse influxdb.csv

In [51]:
def get_first(values):
    first = values[0]
    for v in values:
        if v < first:
            first = v
    
    return first
    
    
def get_last(values):
    last = values[0]
    for v in values:
        if v > last:
            last = v
    
    return last

In [67]:
def get_values(rows, index):
    values = []
    for r in rows:
        values.append(r[index])
    
    return values
    

def get_interval_runtime(rows):
    values = get_values(rows, 3)
    first = get_first(values)
    values = get_values(rows, 5)
    last = get_last(values)
    runtime = round(float(last) - float(first), 3)
    
    return runtime

In [53]:
with open(workdir + "output/influxdb.csv", "r") as influxdb_csv:
    influxdb_content = list(csv.reader(influxdb_csv, delimiter=','))[1:]

In [72]:
csv_header = "blur, mask, dummy, blur+mask, mask+dummy, global"
csv_content = []
csv_checks = []

for i in range(15):
    blur_1 = influxdb_content[i*6]
    blur_2 = influxdb_content[i*6 + 1]
    mask_1 = influxdb_content[i*6 + 2]
    mask_2 = influxdb_content[i*6 + 3]
    dummy_1 = influxdb_content[i*6 + 4]
    dummy_2 = influxdb_content[i*6 + 5]
    
    blur = get_interval_runtime([blur_1, blur_2])
    mask = get_interval_runtime([mask_1, mask_2])
    dummy = get_interval_runtime([dummy_1, dummy_2])
    
    blur_mask = get_interval_runtime([blur_1, blur_2, mask_1, mask_2])
    mask_dummy = get_interval_runtime([mask_1, mask_2, dummy_1, dummy_2])
    global_time = get_interval_runtime([blur_1, blur_2, mask_1, mask_2, dummy_1, dummy_2])
    
    times = [blur, mask, dummy, blur_mask, mask_dummy, global_time]
    
    for i in range(len(times)):
        times[i] = round(times[i], 3)
    
    checks = [0] * 6
    constraints = [30, 30, 30, 50, 50, 60]
    
    for i in range(6):
        if times[i] > constraints[i]:
            checks[i] = 1
    
    csv_checks.append(str(checks))    
    csv_content.append(str(times))

In [71]:
for i in range(15):
    blur_1 = influxdb_content[i*6]
    blur_2 = influxdb_content[i*6 + 1]
    mask_1 = influxdb_content[i*6 + 2]
    mask_2 = influxdb_content[i*6 + 3]
    dummy_1 = influxdb_content[i*6 + 4]
    dummy_2 = influxdb_content[i*6 + 5]
    
    print(blur_1)
    print(blur_2)
    print(mask_1)
    print(mask_2)
    print(dummy_1)
    print(dummy_2)
    
    break

['video_0', 'vnode-2.localdomain', 'blur', '1655131211.386', '1655131226.1276991', '1655131228.3781693']
['video_0', 'vnode-2.localdomain', 'blur', '1655131211.386', '1655131232.2079945', '1655131234.4072266']
['video_0', 'vnode-2.localdomain', 'mask', '1655131235.367', '1655131244.2007', '1655131244.616981']
['video_0', 'vnode-2.localdomain', 'mask', '1655131235.398', '1655131253.8901649', '1655131254.3599198']
['video_0', 'vnode-2.localdomain', 'dummy', '1655131244.774', '1655131263.0724108', '1655131273.6605575']
['video_0', 'vnode-2.localdomain', 'dummy', '1655131254.549', '1655131281.968743', '1655131295.6933305']


In [73]:
with open(workdir + "output/constraints_validation.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_content:
        file.write(row[1:-1] + "\n")
print("Done!")

Done!


In [75]:
with open(workdir + "output/constraints_check.csv", "w") as file:
    file.write(csv_header + "\n")
    for row in csv_checks:
        file.write(row[1:-1] + "\n")
print("Done!")

Done!
